In [1]:
import numpy as np

from keras.preprocessing import sequence
from keras.optimizers import SGD, RMSprop, Adagrad
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Embedding
from keras.layers.recurrent import LSTM, GRU

In [21]:
import json
filename = "/media/charles/data/articles"
h = open(filename)
all_jsons=[]
for line in h:    
    if line[0]=='[':
        all_jsons.append(line[:-1])
        
TitleList = []
#TextList = []
IndexList = []
count = 0

for oneJson in all_jsons:
    u = json.loads(oneJson)
    for item in u:
        fields = item['fields']
        TitleList.append(fields['title'].encode('ascii','ignore'))
        #TextList.append(fields['text'].encode('ascii','ignore'))
        IndexList.append(item['rowKey'].encode('ascii','ignore'))
        count+=1
        if count%10000==0:
            print("done: "+str(count))

all_jsons = []
del all_jsons

done: 10000
done: 20000
done: 30000
done: 40000
done: 50000
done: 60000
done: 70000
done: 80000
done: 90000
done: 100000


In [38]:
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils

max_words = 10000
batch_size = 16

print "Loading data..."
X_seq = TitleList[0:60000]

print len(X_seq), 'total sequences'

print "Vectorizing sequence data..."
tokenizer = Tokenizer(nb_words=max_words)
tokenizer.fit_on_texts(X_seq)
X_seq = tokenizer.texts_to_sequences(X_seq)

print "Divide train and test"
y_train = []
X_train = []
maxlen = 50

for s in X_seq:
    if len(s) < 2 or len(s)>maxlen:
        continue    
    y_train.append(s[-1])
    X_train.append(s[:-2])

y_train = np_utils.to_categorical(y_train, max_words)
    
cut = int(len(X_train)*0.8)
X_test = X_train[cut:]
X_train = X_train[:cut]
y_test = y_train[cut:]
y_train = y_train[:cut]
    
print "Pad sequences (samples x time)"
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)

print 'X_train shape:', X_train.shape
print 'X_test shape:', X_test.shape

#print "Convert class vector to binary class matrix (for use with categorical_crossentropy)"
#Y_train = np_utils.to_categorical(y_train, nb_classes)
#Y_test = np_utils.to_categorical(y_test, nb_classes)
#print 'Y_train shape:', Y_train.shape
#print 'Y_test shape:', Y_test.shape


Loading data...
60000 total sequences
Vectorizing sequence data...
Divide train and test
Pad sequences (samples x time)
X_train shape: (47864, 50)
X_test shape: (11967, 50)


In [34]:
np.max(X_train)

9999

In [41]:
max_features= max_words
emb_size = 256
batch_size = 16

print 'Build model...'
model = Sequential()
model.add(Embedding(max_features, emb_size))
model.add(LSTM(emb_size, 128)) # try using a GRU instead, for fun
#model.add(Dropout(0.5))
model.add(Dense(128, max_features))
model.add(Activation('softmax'))

# try using different optimizers and different optimizer configs
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')


Build model...


In [42]:

print "Train..."
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=10, verbose=0)
score = model.evaluate(X_test, y_test, batch_size=batch_size)
print 'Test score:', score

classes = model.predict_classes(X_test, batch_size=batch_size)
acc = np_utils.accuracy(classes, y_test)
print 'Test accuracy:', acc



Train...


KeyboardInterrupt: 

In [8]:
def tostr(u):
    st = ""
    for x in u:
        st+=voc[x]
    return st

def see(X, Y):
    for x,y in zip (X,Y):
        print tostr(x), voc[y]

def seeModel(X, model):
    see(X, model.predict_classes(X))

In [9]:
seeModel(X_test[0:20], model)

*ek though, ev%%%%%%%%%%%%%%%% e
*iments'he%%%%%%%%%%%%%%%%%%%% o
*'quot'negativ%%%%%%%%%%%%%%%%  
*erness.'lt%%%%%%%%%%%%%%%%%%% '
*and james%%%%%%%%%%%%%%%%%%%%  
*he next %%%%%%%%%%%%%%%%%%%%% t
*t the bottom%%%%%%%%%%%%%%%%%  
*, very c%%%%%%%%%%%%%%%%%%%%% o
*sometimes fresh%%%%%%%%%%%%%%  
'warning'lt''em'gt%%%%%%%%%%%% '
' a question %%%%%%%%%%%%%%%%% t
*to the pan if%%%%%%%%%%%%%%%%  
*oked covered wit%%%%%%%%%%%%%  
* your fish, y%%%%%%%%%%%%%%%% e
*using lemon%%%%%%%%%%%%%%%%%%  
*ref''quo%%%%%%%%%%%%%%%%%%%%% t
* the brow%%%%%%%%%%%%%%%%%%%%  
* in your ici%%%%%%%%%%%%%%%%% n
*hay' is litera%%%%%%%%%%%%%%% n
*ore than%%%%%%%%%%%%%%%%%%%%%  
